In [1]:
import pathlib

# model_file = "../data/mistral-7b-v0.1.Q4_K_M.gguf"
# model_file = "../data/mistral-7b-v0.1.Q8_0.gguf"
model_file = "../data/mistral-7b-instruct-v0.1.Q8_0.gguf"
pathlib.Path(model_file).exists()

True

In [2]:
from llama_cpp import Llama

n_gqa = 8 if "70b" in model_file else 1
llm = Llama(model_path=model_file, n_gqa=n_gqa, n_gpu_layers=16)
llm

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 2 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
  Device 1: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ../data/mistral-7b-instruct-v0.1.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.blo

In [4]:
prompt = """富士山の高さは？正確に"""
# prompt = """クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。"""

prompt_formatted = f"""
<s>[INST]
<<SYS>>あなたは誠実で優秀な日本人のアシスタントです。<</SYS>>
{prompt}
[/INST]
"""

# # NOTE: alpaca format
# prompt_formatted = """### Instruction: What is the height of Mount Fuji?
# ### Response:"""


# 推論の実行
for jsn in llm(
    prompt_formatted.strip(),
    max_tokens=256,
    temperature=0.0001,
    stop=None,
    echo=True,
    stream=True,
):
    print(jsn["choices"][0]["text"], sep="", end="")
print()

Llama.generate: prefix-match hit


 富士山の高さは3,779.24米です。



llama_print_timings:        load time =     897.83 ms
llama_print_timings:      sample time =       7.64 ms /    21 runs   (    0.36 ms per token,  2747.97 tokens per second)
llama_print_timings: prompt eval time =     437.52 ms /     8 tokens (   54.69 ms per token,    18.28 tokens per second)
llama_print_timings:        eval time =    3107.86 ms /    20 runs   (  155.39 ms per token,     6.44 tokens per second)
llama_print_timings:       total time =    3593.61 ms


In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#
#
# model_id = "mistralai/Mistral-7B-v0.1"
# tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
# assert llm.token_bos() == tokenizer.bos_token_id
# assert llm.token_eos() == tokenizer.eos_token_id

In [ ]:
from app.llama2cpp.component.llama2cpp import LlamaCppCustom


n_gqa = 8 if "70b" in model_file else 1
llm = LlamaCppCustom(
    model_path=model_file,
    n_ctx=512,
    temperature=0.0001,
    max_tokens=512,
    n_gqa=n_gqa,
    n_gpu_layers=34,
    verbose=True,
    streaming=True,
    repeat_penalty=1.1,
)

In [ ]:
# prompt = "明日の東京の天気は?"
prompt = "1から20までの間の数で、5で終わる数は？"
prompt_formatted = f"""[INST] <<SYS>>あなたは誠実で優秀な日本人のアシスタントです。<</SYS>> {prompt} [/INST]"""

text = ""
# async for tkn in llm.astream(input=prompt_formatted, stop=None):
for tkn in llm.stream(input=prompt_formatted, stop=None):
    print(tkn, sep="", end="")
    text += tkn
print("")